In [1]:
import pandas as pd
import glob
from ast import literal_eval
files=glob.glob("data/*.json")

In [2]:
l=[]
for filename in files:
    try:
        l.append(pd.read_json(filename, encoding='UTF-8'))
    except:
        next

In [3]:
df = pd.concat(l, axis=0)

In [4]:
cleandf=df.dropna()
cleandf=cleandf.sample(3000)

In [5]:
from pandas.io.json import json_normalize
df_info = json_normalize(cleandf.to_dict('list'), ['educations']).unstack().apply(pd.Series)

In [6]:
education=pd.concat([df_info.groupby(df_info.index.get_level_values(1))[col].apply(lambda x: list(x)) for col in df_info.columns],axis=1)
cleandf=cleandf.reset_index(drop=True)
education=education.drop('end-date',axis=1)
education=education.drop('start-date',axis=1)
education=education.rename(columns={'summary':'positions_summary'})
cleandf=pd.concat([cleandf,education],axis=1)

In [7]:
from pandas.io.json import json_normalize
df_info = json_normalize(cleandf.to_dict('list'), ['positions']).unstack().apply(pd.Series)

positions=pd.concat([df_info.groupby(df_info.index.get_level_values(1))[col].apply(lambda x: list(x)) for col in df_info.columns],axis=1)
cleandf=cleandf.reset_index(drop=True)
positions=positions.drop('end-date',axis=1)
positions=positions.drop('start-date',axis=1)
positions=positions.rename(columns={'summary':'positions_summary'})
cleandf=pd.concat([cleandf,positions],axis=1)

In [8]:
educol=list(education.columns)
poscol=list(positions.columns)

In [9]:
#for col in cleandf.columns:
#    print(col)
#    cleandf[col] = cleandf[col].str.replace('[^\x00-\x7F]','')
#cleandf.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [10]:
poscol.remove('is-current')

In [11]:
for col in educol:
    print(col)
    cleandf[col]=cleandf[col].apply(lambda x: [val for val in x if pd.notnull(val)])
    #cleandf[col]=cleandf[col].apply(lambda x:['"'+v+'"' for v in x])
    
for col in poscol:
    print(col)
    cleandf[col]=cleandf[col].apply(lambda x: [val for val in x if pd.notnull(val)])
    #cleandf[col]=cleandf[col].apply(lambda x:['"'+v+'"' for v in x])

degree
field-of-study
school-name
company-name
positions_summary
title


In [12]:
cleandf.to_pickle("clean_data_3000.pkl")